In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
# This file was saved during the previous assignment before the classifier was run
# Note: Please make sure this python notebook is in the same directory where notMNIST.pickle was saved, or 
# this below code won't work
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
# This code reformats the training, validation and test set data into a format more suited to using TensorFlow
# Firstly, reformat the data itself into a 2D array instead of 3D. This is done essentially by collapsing the 
# X and Y data together [so instead of a 2D 28*28 array for X and Y, make a 28^2 1D array representing the image]
# Also, turn the labels into a 1 hot encoding instead of the format from before. Essentially now, instead of 
# there only being a 1D labels array with the correct label at a relevant index, there will be a 2D array
# with a one-hot vector representing the label at the relevant index.
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
# This code initializes a computational graph for this data in TensorFlow. 

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.314758
Training accuracy: 12.8%
Validation accuracy: 14.5%
Loss at step 100: 2.230618
Training accuracy: 71.7%
Validation accuracy: 70.6%
Loss at step 200: 1.802410
Training accuracy: 74.7%
Validation accuracy: 72.8%
Loss at step 300: 1.573099
Training accuracy: 76.2%
Validation accuracy: 73.8%
Loss at step 400: 1.418737
Training accuracy: 77.2%
Validation accuracy: 74.0%
Loss at step 500: 1.303020
Training accuracy: 77.9%
Validation accuracy: 74.5%
Loss at step 600: 1.211169
Training accuracy: 78.4%
Validation accuracy: 74.8%
Loss at step 700: 1.135854
Training accuracy: 78.8%
Validation accuracy: 75.1%
Loss at step 800: 1.072720
Training accuracy: 79.3%
Validation accuracy: 75.2%
Test accuracy: 82.2%


In [10]:
# The below code re-initializes the TensorFlow graph for training with stochastic gradient descent instead of
# simple gradient descent. Instead of a constant, the training set is now random placeholder data. Real 
# data is fed into the model during training on each call to session.run()

# This batch size represents how many rows of the total data table are fed into the network for training at once
# We generally use it to train the network in chunks because memory usually runs out
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [11]:
# This code actually runs stochastic gradient descent on the above initialized graph.
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.363358
Minibatch accuracy: 11.7%
Validation accuracy: 18.8%
Minibatch loss at step 500: 1.288315
Minibatch accuracy: 80.5%
Validation accuracy: 74.6%
Minibatch loss at step 1000: 1.651031
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 1500: 1.048665
Minibatch accuracy: 78.1%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.918928
Minibatch accuracy: 80.5%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 1.203052
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.539511
Minibatch accuracy: 85.9%
Validation accuracy: 77.9%
Test accuracy: 85.4%


In [13]:
# Problem

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_in must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
  # The shape of weights_out must be the same as the number of columns of weights_in times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This was made into a function because this series of operations is performed
  # multiple times in addition to trianing when calculating predictions for the validation and test 
  # sets
  def training_comps(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes layer_1 to be based on ReLU's
      out_layer = tf.matmul(in_layer, out_weights) + out_biases
      return out_layer
  
  logits = training_comps(tf_train_dataset, weights_in, biases_in, weights_out, biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, weights_in, biases_in, weights_out, biases_out))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, weights_in, biases_in, weights_out, biases_out))

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 293.486755
Minibatch accuracy: 13.3%
Validation accuracy: 23.4%
Minibatch loss at step 500: 11.801166
Minibatch accuracy: 85.2%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 14.771784
Minibatch accuracy: 77.3%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 5.598125
Minibatch accuracy: 78.1%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 2.267886
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 10.165213
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 1.734764
Minibatch accuracy: 89.1%
Validation accuracy: 81.5%
Test accuracy: 87.7%
